### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset and create dataframe  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
complete_df = school_data_complete

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Determine Total Schools via number of unique (.nunique)school_names
tot_schools = complete_df["school_name"].nunique()

# Determine Total Students via .count of student_name
tot_students = complete_df["student_name"].count()

# Determine Total Budget via sum of .unique budget and convert to float
tot_budget = (sum(complete_df["budget"].unique())).astype(float)

# Determine Average Math and Reading Score via .mean of math_score and reading_score respectively
avg_math = complete_df["math_score"].mean()
avg_reading = complete_df["reading_score"].mean()

# Determine % Passing Math & Reading via (sum * 100) of greater than passing grade (> 69) in math_score and reading_score respectively / by tot_students
pass_check_math = complete_df["math_score"] > 69
pass_math = (sum(pass_check_math)/tot_students)*100
pass_check_reading = complete_df["reading_score"] > 69
pass_reading = (sum(pass_check_reading)/tot_students)*100

# Determine % Overall Passing via (sum * 100) of greater than passing grade (> 69) in both math_score and reading_score / by tot_students
pass_check_over = (complete_df["math_score"] > 69) & (complete_df["reading_score"] > 69)
pass_over = (sum(pass_check_over)/tot_students)*100

# Create District Summary dataframe using lists determined above
dist_summary_df = pd.DataFrame({"Total Schools": tot_schools,
                           "Total Students": tot_students,
                           "Total Budget": tot_budget,
                           "Average Math Score": avg_math,
                           "Average Reading Score": avg_reading,
                           "% Passing Math": pass_math,
                           "% Passing Reading": pass_reading,
                           "% Overall Passing": pass_over
                          }, index=[0])

# format Total Students and Total Budget columns accordingly
dist_summary_df["Total Students"] = dist_summary_df["Total Students"].map("{:,}".format)
dist_summary_df["Total Budget"] = dist_summary_df["Total Budget"].map("${:,.2f}".format)
dist_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# Sort by School Name
complete_df = complete_df.sort_values("school_name")

# Reduce irrelevant columns and drop duplicates of school_name column for ease of use going forward
schools_sorted_df = complete_df[["Student ID", "school_name", "reading_score", "math_score", "type", "budget"]\
                               ].drop_duplicates(subset='school_name', keep="first")

In [4]:
# Determine list of School Name
school_names = schools_sorted_df["school_name"]

# Determine list of School Type and index with school_names
school_type = schools_sorted_df["type"]
school_type.index = school_names

# Determine list of Total Students per school via .value_counts of school_name from complete_df, prevent automatic sort of series
school_tot_students = complete_df["school_name"].value_counts(sort=False)

In [9]:
# Determine list of Total School Budget and index with school_names
school_tot_budget = schools_sorted_df["budget"]
school_tot_budget.index = school_names

# Determine list of Per Student Budget via division of school_tot_budget by school_tot_students
student_budget = school_tot_budget/school_tot_students

In [10]:
# Create dataframe grouped by school_name
grouped_schools = complete_df.groupby(["school_name"])

# Determine Average Math and Reading Score via .mean of math_score and reading_score respectively
avg_math = grouped_schools["math_score"].mean()
avg_reading = grouped_schools["reading_score"].mean()

In [11]:
# Get total number of Students Passing Math, Reading & Overall per school and create corresponding columns using (> 69, .groupby of school_name & sum)
complete_df["Students Passing Math"] = (complete_df["math_score"] > 69).groupby(complete_df["school_name"]).transform('sum')
complete_df["Students Passing Reading"] = (complete_df["reading_score"] > 69).groupby(complete_df["school_name"]).transform('sum')
complete_df["Students Passing Overall"] = ((complete_df["math_score"] > 69) & (complete_df["reading_score"] > 69))\
.groupby(complete_df["school_name"]).transform('sum')

# Determine % Passing Math, Reading & Overall by reducing totals to unique schools, converting to integer & / by tot_students before * 100
pass_studmath = (((complete_df["Students Passing Math"]).groupby(complete_df["school_name"]).unique()\
                 ).astype(int)/school_tot_students)*100
pass_studreading = (((complete_df["Students Passing Reading"]).groupby(complete_df["school_name"]).unique()\
                    ).astype(int)/school_tot_students)*100
pass_studover = (((complete_df["Students Passing Overall"]).groupby(complete_df["school_name"]).unique()\
                 ).astype(int)/school_tot_students)*100

In [45]:
# Create School Summary dataframe using determined lists above
school_summary_df = pd.DataFrame({"School Type": school_type,
                                  "Total Students": school_tot_students,
                                  "Total School Budget": school_tot_budget,
                                  "Per Student Budget": student_budget,
                                  "Average Math Score": avg_math,
                                  "Average Reading Score": avg_reading,
                                  "% Passing Math": pass_studmath,
                                  "% Passing Reading": pass_studreading,
                                  "% Overall Passing": pass_studover
                                 })

# Format Total School Budget and Per Student Budget columns accordingly
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [10]:
# Sort and display the top five performing schools by % Overall Passing
top5overall = school_summary_df.sort_values("% Overall Passing", ascending=False)
top5overall.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [11]:
# Sort and display the five worst-performing schools by % Overall Passing
bot5overall = school_summary_df.sort_values("% Overall Passing")
bot5overall.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [12]:
# Create table with .groupby function grouped by school_name and grade; determine and add columns of .mean of math_score and reading_score
grouped_schoolgrades = complete_df.groupby(["school_name", "grade"]).agg({"math_score": ['mean'], "reading_score": ['mean']})
grouped_schoolgrades.columns = ["math score per grade", "reading score per grade"]

# Reset index of table
grouped_schoolgrades = grouped_schoolgrades.reset_index()

In [13]:
# Determine lists of grade specific math score per school with conditional and set school_names as index of each
mnines = grouped_schoolgrades.loc[grouped_schoolgrades["grade"] == "9th"].iloc[:, 2]
mnines.index = school_names

mtens = grouped_schoolgrades.loc[grouped_schoolgrades["grade"] == "10th"].iloc[:, 2]
mtens.index = school_names

melevens = grouped_schoolgrades.loc[grouped_schoolgrades["grade"] == "11th"].iloc[:, 2]
melevens.index = school_names

mtwelves = grouped_schoolgrades.loc[grouped_schoolgrades["grade"] == "12th"].iloc[:, 2]
mtwelves.index = school_names

In [14]:
# Create Grade Math Score Summary dataframe using determined lists above
mathbygrade_df = pd.DataFrame({"9th": mnines,
                              "10th": mtens,
                              "11th": melevens,
                              "12th": mtwelves
                             })
mathbygrade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [20]:
# Determine lists of grade specific reading score per school with conditional and set school_names as index of each
rnines = grouped_schoolgrades.loc[grouped_schoolgrades["grade"] == "9th"].iloc[:, 3]
rnines.index = school_names

rtens = grouped_schoolgrades.loc[grouped_schoolgrades["grade"] == "10th"].iloc[:, 3]
rtens.index = school_names

relevens = grouped_schoolgrades.loc[grouped_schoolgrades["grade"] == "11th"].iloc[:, 3]
relevens.index = school_names

rtwelves = grouped_schoolgrades.loc[grouped_schoolgrades["grade"] == "12th"].iloc[:, 3]
rtwelves.index = school_names

# Create Grade Reading Score Summary dataframe using determined lists above
readingbygrade_df = pd.DataFrame({"9th": rnines,
                                 "10th": rtens,
                                 "11th": relevens,
                                 "12th": rtwelves
                                })
readingbygrade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [47]:
# Create bins in which to place values based upon Per Student Budget and corresponding labels
budgetbins = [0, 585, 630, 645, 680]
budgetbin_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Slice the data and place it into bins, minding to convert Per Student Budget to float
scoresby_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Student Budget"].astype(float), budgetbins, labels=budgetbin_labels)

In [48]:
# Determine mean of new groupings in Average Math and Reading Score, % Passing Math, Reading and Overall via .mean and .groupby function
budgetgroups_df = scoresby_df.groupby(["Spending Ranges (Per Student)"]).agg({"Average Math Score": ['mean'], "Average Reading Score": ['mean'], "% Passing Math": ['mean'], "% Passing Reading": ['mean'], "% Overall Passing": ['mean']})
budgetgroups_df.columns = ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

# Create column for Spending Ranges (Per Student) from bin_labels used earlier and set index to this column
budgetgroups_df["Spending Ranges (Per Student)"] = budgetbin_labels 
budgetgroups_df = budgetgroups_df.set_index(["Spending Ranges (Per Student)"])

# Format all columns within dataframe to two decimal points
budgetgroups_df.loc[:, :] = budgetgroups_df.loc[:, :].applymap("{:,.2f}".format)
budgetgroups_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [49]:
# Create bins in which to place values based upon School Size and corresponding labels
sizebins = [0, 1000, 2000, 5000]
sizebin_labels = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

# Slice the data and place it into bins
sizegroups_df["School Size"] = pd.cut(school_summary_df["Total Students"], sizebins, labels=sizebin_labels)

# Determine mean of new groupings in Average Math and Reading Score, % Passing Math, Reading and Overall via .mean and .groupby function
sizegroups_df = sizegroups_df.groupby(["School Size"]).agg({"Average Math Score": ['mean'], "Average Reading Score": ['mean'], "% Passing Math": ['mean'], "% Passing Reading": ['mean'], "% Overall Passing": ['mean']})
sizegroups_df.columns = ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

# Create column for School Size from bin_labels used earlier and re-index dataframe to use this column as index
sizegroups_df["School Size"] = sizebin_labels 
sizegroups_df = sizegroups_df.set_index(["School Size"])

sizegroups_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium(1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large(2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [50]:
# Replace Charter and District in School Type column with 1 and 2 respectively to allow binning
typegroups_df = school_summary_df.replace(["Charter", "District"], [1, 2])

# Create bins in which to place values based upon School Type and corresponding labels
typebins = [0, 1.1, 2.1]
typebin_labels = ["Charter", "District"]

# Slice the data and place it into bins
typegroups_df["Type"] = pd.cut(typegroups_df["School Type"], typebins, labels=typebin_labels)

# Determine mean of new groupings in Average Math and Reading Score, % Passing Math, Reading and Overall via .mean and .groupby function
typegroups_df = typegroups_df.groupby(["School Type"]).agg({"Average Math Score": ['mean'], "Average Reading Score": ['mean'], "% Passing Math": ['mean'], "% Passing Reading": ['mean'], "% Overall Passing": ['mean']})
typegroups_df.columns = ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

# Create column for School Type from bin_labels used earlier and re-index dataframe to use this column as index
typegroups_df["School Type"] = typebin_labels 
typegroups_df = typegroups_df.set_index(["School Type"])

typegroups_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
